In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!git pull

Already up to date.


In [3]:
! pip install mlflow transformers datasets "pydantic[dotenv]" av

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 91.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 57.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 kB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 kB 96.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.7/232.7 kB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.

In [4]:
import mlflow
from mlflow.client import MlflowClient
import pandas as pd
import os
import matplotlib.pyplot as plt
from overfit.env_settings import settings
from overfit.utils.mlflow import get_all_experiments, get_params_from_experiment_name
import numpy as np
import itertools
from tqdm.notebook import tqdm


In [ ]:
assert settings.MLFLOW_TRACKING_URI


In [6]:
client = MlflowClient(tracking_uri=settings.MLFLOW_TRACKING_URI)


In [7]:
ALL_CONFIDENCES = [0.1, 0.2, 0.3, 0.9]
ALL_WEIGHT_DECAYS = [0.1, 0.2, 0.3, 0.4]
ALL_MAX_LRS = [0.1, 0.2, 0.3, 0.4]
ALL_DATSETS = ["4-50", "5-50", "6-75"]
MODELS = ["resnet18", "resnet34", "resnet50", "vit"]


In [8]:
CONFIDENCES = [0.1]
WEIGHT_DECAYS = [0.2]
MAX_LRS = [0.1, 0.2, 0.3, 0.4]
MOMENTUMS = [0.1]
DATASETS = ["4-50"]
MODELS = ["vit"]

EXPERIMENT_NAMES = get_all_experiments(
    datasets=DATASETS,
    models=MODELS,
    confidences=CONFIDENCES,
    weight_decays=WEIGHT_DECAYS,
    max_lrs=MAX_LRS,
    momentums=MOMENTUMS,
)
EXPERIMENT_NAMES


['D4-50MvitC0.1WD0.2LR0.1M0.1',
 'D4-50MvitC0.1WD0.2LR0.2M0.1',
 'D4-50MvitC0.1WD0.2LR0.3M0.1',
 'D4-50MvitC0.1WD0.2LR0.4M0.1']

In [9]:
experiments = [
    client.get_experiment_by_name(experiment_name)
    for experiment_name in tqdm(EXPERIMENT_NAMES)
]


  0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
experiments


[<Experiment: artifact_location='mlflow-artifacts:/41', creation_time=1669169218111, experiment_id='41', last_update_time=1669169218111, lifecycle_stage='active', name='D4-50MvitC0.1WD0.2LR0.1M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/42', creation_time=1669169218286, experiment_id='42', last_update_time=1669169218286, lifecycle_stage='active', name='D4-50MvitC0.1WD0.2LR0.2M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/43', creation_time=1669169218414, experiment_id='43', last_update_time=1669169218414, lifecycle_stage='active', name='D4-50MvitC0.1WD0.2LR0.3M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/32', creation_time=1669040748038, experiment_id='32', last_update_time=1669040748038, lifecycle_stage='active', name='D4-50MvitC0.1WD0.2LR0.4M0.1', tags={}>]

In [15]:
def get_experiment_error_df(experiment_id: str) -> pd.DataFrame:
    src_errors = []
    tgt_errors = []
    runs = client.search_runs(experiment_ids=[experiment_id])
    for run in tqdm(runs):
        N = int(run.data.params["Frames"])
        filename = run.data.params["Filename"]
        src_acc_top1 = run.data.metrics["Source Accumulated Top-1 Count"]
        tgt_acc_top1 = run.data.metrics["Target Accumulated Top-1 Count"]
        src_error = 100 * (N - src_acc_top1) / N
        tgt_error = 100 * (N - tgt_acc_top1) / N
        assert src_error >= 0, (src_error, src_acc_top1, N, filename)
        assert tgt_error >= 0, (tgt_error, tgt_acc_top1, N, filename)
        src_errors.append(src_error)
        tgt_errors.append(tgt_error)

    df = pd.DataFrame(
        {
            "Source Error": src_errors,
            "Target Error": tgt_errors,
        }
    )
    return df.describe()


In [16]:
experiments

[<Experiment: artifact_location='mlflow-artifacts:/41', creation_time=1669169218111, experiment_id='41', last_update_time=1669169218111, lifecycle_stage='active', name='D4-50MvitC0.1WD0.2LR0.1M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/42', creation_time=1669169218286, experiment_id='42', last_update_time=1669169218286, lifecycle_stage='active', name='D4-50MvitC0.1WD0.2LR0.2M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/43', creation_time=1669169218414, experiment_id='43', last_update_time=1669169218414, lifecycle_stage='active', name='D4-50MvitC0.1WD0.2LR0.3M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/32', creation_time=1669040748038, experiment_id='32', last_update_time=1669040748038, lifecycle_stage='active', name='D4-50MvitC0.1WD0.2LR0.4M0.1', tags={}>]

In [17]:
dfs = [
    get_experiment_error_df(experiment.experiment_id)
    for experiment in tqdm(experiments)
]


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/976 [00:00<?, ?it/s]

  0%|          | 0/976 [00:00<?, ?it/s]

  0%|          | 0/976 [00:00<?, ?it/s]

  0%|          | 0/976 [00:00<?, ?it/s]

In [18]:
experiment_names = [experiment.name for experiment in experiments]


In [19]:
params_df = pd.DataFrame(
    [
        get_params_from_experiment_name(experiment_name)
        for experiment_name in experiment_names
    ],
    index=experiment_names,
)
params_df

,dataset,model,confidence,weight_decay,max_lr,momentum
D4-50MvitC0.1WD0.2LR0.1M0.1,4-50,vit,0.1,0.2,0.1,0.1
D4-50MvitC0.1WD0.2LR0.2M0.1,4-50,vit,0.1,0.2,0.2,0.1
D4-50MvitC0.1WD0.2LR0.3M0.1,4-50,vit,0.1,0.2,0.3,0.1
D4-50MvitC0.1WD0.2LR0.4M0.1,4-50,vit,0.1,0.2,0.4,0.1


In [20]:
mean_df = pd.concat([df.loc["mean"] for df in dfs], axis=1).T
mean_df = mean_df.set_axis(
    [experiment.name for experiment in experiments], axis=0
).dropna()
mean_df = mean_df.rename(columns={"Source Error": "Source Error Mean", "Target Error": "Target Error Mean"})
mean_df_base = mean_df[["Source Error Mean", "Target Error Mean"]]
mean_df


,Source Error Mean,Target Error Mean
D4-50MvitC0.1WD0.2LR0.1M0.1,59.351015,57.017511
D4-50MvitC0.1WD0.2LR0.2M0.1,59.351015,55.598687
D4-50MvitC0.1WD0.2LR0.3M0.1,59.351015,54.612751
D4-50MvitC0.1WD0.2LR0.4M0.1,59.351015,53.890648


In [21]:
mean_df["Error Mean Percentual Change"] = (
    100
    * (mean_df["Target Error Mean"] - mean_df["Source Error Mean"])
    / mean_df["Source Error Mean"]
)
# mean_df["Error Mean Absolute Change"] = mean_df["Target Error Mean"] - mean_df["Source Error Mean"]
mean_df_change = mean_df[["Error Mean Percentual Change"]]
mean_df


,Source Error Mean,Target Error Mean,Error Mean Percentual Change
D4-50MvitC0.1WD0.2LR0.1M0.1,59.351015,57.017511,-3.931700
D4-50MvitC0.1WD0.2LR0.2M0.1,59.351015,55.598687,-6.322265
D4-50MvitC0.1WD0.2LR0.3M0.1,59.351015,54.612751,-7.983459
D4-50MvitC0.1WD0.2LR0.4M0.1,59.351015,53.890648,-9.200124


In [22]:
std_df = pd.concat([df.loc["std"] for df in dfs], axis=1).T
std_df = std_df.set_axis(
    [experiment.name for experiment in experiments], axis=0
).dropna()
std_df = std_df.rename(columns={"Source Error": "Source Error Std", "Target Error": "Target Error Std"})
std_df_base = std_df[["Source Error Std", "Target Error Std"]]
std_df

,Source Error Std,Target Error Std
D4-50MvitC0.1WD0.2LR0.1M0.1,23.749646,24.915604
D4-50MvitC0.1WD0.2LR0.2M0.1,23.749646,25.472482
D4-50MvitC0.1WD0.2LR0.3M0.1,23.749646,25.828909
D4-50MvitC0.1WD0.2LR0.4M0.1,23.749646,26.057293


In [23]:
std_df["Error Std Percentual Change"] = (
    100
    * (std_df["Target Error Std"] - std_df["Source Error Std"])
    / std_df["Source Error Std"]
)
# std_df["Error Std Absolute Change"] = std_df["Target Error Std"] - std_df["Source Error Std"]
std_df_change = std_df[["Error Std Percentual Change"]]
std_df


,Source Error Std,Target Error Std,Error Std Percentual Change
D4-50MvitC0.1WD0.2LR0.1M0.1,23.749646,24.915604,4.909367
D4-50MvitC0.1WD0.2LR0.2M0.1,23.749646,25.472482,7.254153
D4-50MvitC0.1WD0.2LR0.3M0.1,23.749646,25.828909,8.754919
D4-50MvitC0.1WD0.2LR0.4M0.1,23.749646,26.057293,9.716553


In [24]:
mean_df.join(std_df)

,Source Error Mean,Target Error Mean,Error Mean Percentual Change,Source Error Std,Target Error Std,Error Std Percentual Change
D4-50MvitC0.1WD0.2LR0.1M0.1,59.351015,57.017511,-3.931700,23.749646,24.915604,4.909367
D4-50MvitC0.1WD0.2LR0.2M0.1,59.351015,55.598687,-6.322265,23.749646,25.472482,7.254153
D4-50MvitC0.1WD0.2LR0.3M0.1,59.351015,54.612751,-7.983459,23.749646,25.828909,8.754919
D4-50MvitC0.1WD0.2LR0.4M0.1,59.351015,53.890648,-9.200124,23.749646,26.057293,9.716553


In [26]:
all_df = params_df[["max_lr"]].join(mean_df).join(std_df)
all_df.to_latex(float_format="%.2f")


/tmp/ipykernel_411/596001511.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  all_df.to_latex(float_format="%.2f")


'\\begin{tabular}{lrrrrrrr}\n\\toprule\n{} &  max\\_lr &  Source Error Mean &  Target Error Mean &  Error Mean Percentual Change &  Source Error Std &  Target Error Std &  Error Std Percentual Change \\\\\n\\midrule\nD4-50MvitC0.1WD0.2LR0.1M0.1 &    0.10 &              59.35 &              57.02 &                         -3.93 &             23.75 &             24.92 &                         4.91 \\\\\nD4-50MvitC0.1WD0.2LR0.2M0.1 &    0.20 &              59.35 &              55.60 &                         -6.32 &             23.75 &             25.47 &                         7.25 \\\\\nD4-50MvitC0.1WD0.2LR0.3M0.1 &    0.30 &              59.35 &              54.61 &                         -7.98 &             23.75 &             25.83 &                         8.75 \\\\\nD4-50MvitC0.1WD0.2LR0.4M0.1 &    0.40 &              59.35 &              53.89 &                         -9.20 &             23.75 &             26.06 &                         9.72 \\\\\n\\bottomrule\n\\end{tabular

In [27]:
all_df_base = params_df[["max_lr"]].join(mean_df_base).join(std_df_base)
all_df_base

,max_lr,Source Error Mean,Target Error Mean,Source Error Std,Target Error Std
D4-50MvitC0.1WD0.2LR0.1M0.1,0.1,59.351015,57.017511,23.749646,24.915604
D4-50MvitC0.1WD0.2LR0.2M0.1,0.2,59.351015,55.598687,23.749646,25.472482
D4-50MvitC0.1WD0.2LR0.3M0.1,0.3,59.351015,54.612751,23.749646,25.828909
D4-50MvitC0.1WD0.2LR0.4M0.1,0.4,59.351015,53.890648,23.749646,26.057293


In [28]:
all_df_change = params_df[["max_lr"]].join(mean_df_change).join(std_df_change)
all_df_change

,max_lr,Error Mean Percentual Change,Error Std Percentual Change
D4-50MvitC0.1WD0.2LR0.1M0.1,0.1,-3.931700,4.909367
D4-50MvitC0.1WD0.2LR0.2M0.1,0.2,-6.322265,7.254153
D4-50MvitC0.1WD0.2LR0.3M0.1,0.3,-7.983459,8.754919
D4-50MvitC0.1WD0.2LR0.4M0.1,0.4,-9.200124,9.716553


In [11]:
def find_worse(better_id: str, worse_id: str):
    runs = client.search_runs(experiment_ids=[better_id, worse_id], max_results=2000)
    ekey = lambda run: run.info.experiment_id
    runs.sort(key=ekey)

    experiment_runs = dict()
    for k, group in itertools.groupby(runs, key=ekey):
        experiment_runs[k] = {run.data.params["Filename"]: run for run in group}

    metric = "Target Accumulated Top-1 Count"
    worsened = []
    for filename, run in experiment_runs[worse_id].items():
        if (
            experiment_runs[better_id][filename].data.metrics[metric]
            < run.data.metrics[metric]
        ):
            worsened.append(filename)
    
    rows = []
    for filename in worsened:
        baseline = "Source Accumulated Top-1 Count"
        metric = "Target Accumulated Top-1 Count"

        row = {
            "Filename": filename,
            baseline: experiment_runs[worse_id][filename].data.metrics[baseline],
            f"{worse_id}-{metric}": experiment_runs[worse_id][filename].data.metrics[metric],
            f"{better_id}-{metric}": experiment_runs[better_id][filename].data.metrics[metric],
        }
        rows.append(row)

    return pd.DataFrame(rows)
            

In [12]:
experiments

[<Experiment: artifact_location='mlflow-artifacts:/41', creation_time=1669169218111, experiment_id='41', last_update_time=1669169218111, lifecycle_stage='active', name='D4-50MvitC0.1WD0.2LR0.1M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/42', creation_time=1669169218286, experiment_id='42', last_update_time=1669169218286, lifecycle_stage='active', name='D4-50MvitC0.1WD0.2LR0.2M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/43', creation_time=1669169218414, experiment_id='43', last_update_time=1669169218414, lifecycle_stage='active', name='D4-50MvitC0.1WD0.2LR0.3M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/32', creation_time=1669040748038, experiment_id='32', last_update_time=1669040748038, lifecycle_stage='active', name='D4-50MvitC0.1WD0.2LR0.4M0.1', tags={}>]

In [13]:
df = find_worse(better_id="32", worse_id="42")
df

,Filename,Source Accumulated Top-1 Count,42-Target Accumulated Top-1 Count,32-Target Accumulated Top-1 Count
0,/datasets/imagenet1k-4-50/979-valley-4-40.mp4,24.0,21.0,20.0
1,/datasets/imagenet1k-4-50/973-coral_reef-4-40.mp4,29.0,31.0,30.0
2,/datasets/imagenet1k-4-50/970-alp-4-40.mp4,7.0,3.0,2.0
3,/datasets/imagenet1k-4-50/969-eggnog-4-40.mp4,26.0,28.0,27.0
4,/datasets/imagenet1k-4-50/966-red_wine-4-40.mp4,5.0,1.0,0.0
5,/datasets/imagenet1k-4-50/946-cardoon-4-40.mp4,32.0,33.0,32.0
6,/datasets/imagenet1k-4-50/941-acorn_squash-4-4...,5.0,2.0,1.0
7,/datasets/imagenet1k-4-50/94-hummingbird-4-40.mp4,21.0,23.0,22.0
8,/datasets/imagenet1k-4-50/887-vestment-4-40.mp4,5.0,4.0,2.0
9,/datasets/imagenet1k-4-50/876-tub-4-40.mp4,2.0,1.0,0.0
